In [ ]:
def get_channels_info(youtube, channels):
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channels)
    )
    
    ch_response = request.execute()
    JSON(ch_response)
    for channel_item in ch_response['items']:
        ch_data = { 
            'channel_name' : channel_item['snippet']['title'] ,
            'channel_subs' : channel_item['statistics']['subscriberCount'],
            'channel_total_views' : channel_item['statistics']['viewCount'],
            'channel_total_videos' : channel_item['statistics']['videoCount'],
            'channel_playlist_id' : channel_item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(ch_data)
   
    result  = pd.DataFrame(all_data)
    result.index = result.index + 1 
    return result 
   
    

In [ ]:
def get_vids_ids1(youtube,channel_playlist_id):
    next_page_token = None
    vid_list= []
    while True:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                maxResults=50,
                playlistId=channel_playlist_id,
                pageToken=next_page_token
            )
        vid_response = request.execute()
        JSON(vid_response)
      
        next_page_token= vid_response.get('nextPageToken')
   
        for vid_item in vid_response['items']:
                video_id = vid_item['contentDetails']['videoId']
                vid_list.append(video_id)
        
        if next_page_token== None:
            #JSON(vid_response)
            break
    #print(len(vid_list))
    return  vid_list 
    
        
 

In [ ]:
def get_vid_info(youtube,v_ids):
    all_vid_info_list = []
    for i in range(0, len(v_ids),50):
        vid_info_req= youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(v_ids[i:i+50])
        )
        vid_info_response = vid_info_req.execute()
        #JSON(vid_info_response)
        for vid in vid_info_response['items']:
            stats_to_save = { 
                    'snippet':['channelTitle','channelId','title','publishedAt','description','tags','liveBroadcastContent'],
                'contentDetails':['duration','definition','caption'],
                'statistics':['viewCount','likeCount','commentCount']
            } 
            video_info={}
            video_info['video_id'] = vid['id']
            for key in stats_to_save.keys():
                for value in stats_to_save[key]:
                    try:   
                        video_info[value] = vid[key][value]
                    except:
                        video_info[value] = None
            all_vid_info_list.append(video_info)
            rsult = pd.DataFrame(all_vid_info_list)
            rsult.index  = rsult.index +1
    return rsult

    
        

In [ ]:
def get_comments1(youtube, v_ids):
    all_video_comments = []
    for v_id in v_ids:
        try:
            next_page_token = None
            vid_comments = {}
            v_com=[]
            while True:

                comment_request = youtube.commentThreads().list(
                        part="snippet,replies",
                        videoId=v_id,
                        pageToken=next_page_token
                    )
                comment_response = comment_request.execute()
                next_page_token= comment_response.get('nextPageToken')
                #print(next_page_token)


                for c_resp in comment_response['items']:
                    com = c_resp['snippet']['topLevelComment']['snippet']['textOriginal']
                    v_com.append(com)


                if next_page_token== None:
                    #JSON(vid_response)
                    vid_comments['Video_id'] = v_id
                    vid_comments['comment'] = v_com
                    all_video_comments.append(vid_comments)   
                    break
        except :
            #print("Encountered http error")
            pass
            

             
    return pd.DataFrame(all_video_comments)

In [4]:
jupyter nbconvert --to script main_file.ipynb

#from main_file.ipynb import provehwrong

SyntaxError: invalid syntax (431479923.py, line 1)